In [1]:
from qiskit import *#pulls all necessary modules from the base qiskit
import pickle #module that we will use to save the data obtained from the experiment in a txt file

from qiskit.providers.aer import AerSimulator#module to run the circuit in a simulator
from qiskit.providers.ibmq import IBMQ #module to access IBM providers
from qiskit.tools.visualization import plot_histogram#tool to visualize histograms
from qiskit.tools.monitor import job_monitor#tool to view your position in the queue
from qiskit.providers.fake_provider import FakePerth

In [ ]:
FakeBackend = FakePerth()#an exemple of fake provider
simulator = Aer.get_backend('qasm_simulator')#an exemple of simulator

shots = 1024 #number of executions

In [ ]:
IBMQ.load_account()#Load your IBM account to gain access to real IBM providers

provider = IBMQ.get_provider('ibm-q')
Perth = provider.get_backend('ibm_perth')#an exemple of real provider

In [ ]:
def oracle(n):
    qc = QuantumCircuit(n)
    
    #put an oracle of your choice here 

    #visual way to represent this circuit as an operator in the circuit:
    oracle_gate = qc.to_gate()
    oracle_gate.name = 'U$_w$'

    return oracle_gate

In [ ]:
#General expression of the diffuser operator:
def diffuser(n):
    dif_qc = QuantumCircuit(n)
    
    for i in range(n):
        dif_qc.h(i)
    
    for i in range(n):
        dif_qc.x(i)
        
    dif_qc.h(n-1)
    dif_qc.mct(list(range(n-1)),n-1)
    dif_qc.h(n-1)
    
    for i in range(n):
        dif_qc.x(i)
    
    for i in range(n):
        dif_qc.h(i)
        
    dif_gate = dif_qc.to_gate()
    dif_gate.name = "U$_s$"
    
    return dif_gate

In [ ]:
#function that joins all operations in a circuit corresponding to the grover operator:
def circuit(oracle,diffuser,n,m):
    circ = QuantumCircuit(n,n)
    
    for i in range(n):
        circ.h(i)
    
    N = 1
    while N <= m:
        N += 1
        circ.append(oracle, range(n))
        circ.append(diffuser, range(n))
        circ.barrier()
    
    for i in range(n):
        circ.measure(i,i)
        
    return circ

In [ ]:
oracle_gate = oracle_state(n)#put a number of q-bits
dif_gate = diffuser(n)
circ = circuit(oracle_gate,dif_gate,n,m)#the number m is the Grover operation number
circ.draw('mpl')#draw the circuit

In [ ]:
#plotting simulated results from qasm backend:
result = execute(circ, backend = simulator,shots).result()
counts_QASM = result.get_counts(circ)
plot_histogram(counts_QASM)

In [ ]:
#plotting results from fake backend:
result_fake = execute(circ, backend = FakeBackend,shots).result()
counts_fake = result_fake.get_counts(circ)
plot_histogram(counts_fake)

In [ ]:
job = execute(circ, backend= Perth, shots)#execute the circuit in a quantum computer of your choice

job_monitor(job)#way to view your position in the queue

In [ ]:
#plotting results from quantum device
counts_comp = job.result().get_counts()

plot_histogram(counts_comp)

In [ ]:
#method to save quantum device histogram data
hist_save = dict(counts)

file = open('enter de file path','wb')

pickle.dump(hist_save, file)

file.close()